In [1]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.charts import Histogram, Donut
from bokeh.io import output_notebook
from bokeh.models.formatters import DatetimeTickFormatter as dttf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

In [2]:
output_notebook() #loading bokeh for notebook output, will change to file for later dashboard
# output_file("test.html", title = 'test for output')

Loading BokehJS ...

In [3]:
df=pd.read_json('teamsdone.json')

In [4]:
#gathers the total number of SC2 Tournaments Played
df['totalsc2'] = df.groupby('PlayerId')['PlayerId'].transform('count')

In [5]:
#popped ('DOB' Column for now, as it returned improper values)
df.DOB = pd.to_datetime(df.DOB)
df.EndDate = pd.to_datetime(df.EndDate)

In [203]:
df['date']=df['EndDate']
df.set_index('date', drop=True)
df

,Age_at_win,CountryCode,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,GameId,NameFirst,NameLast,Note,PlayerId,Prize,Prize_USD,RankText,TeamPlayers,Team_On,TotalUSDPrize,TournamentName,team_history,totalsc2,AveragePrizePerPlayer,country,currency,Team_On_Backup,teams,Rank,num_teams,date
0,25.0,kr,KRW,MC,1991-06-17,2016-09-10,0.000905,151,Min Chul,Jang,None,1000,3000000.0,2714,17th-24th,1,CJ Entus,506645.42,GSL Season 2 2016 (Code S),"[{'team': 'CJ Entus', 'history': [146396160000...",108,4709.639439,kr,KRW,CJ Entus,CJ Entus,Cash Only,5.0,2016-09-10
1,24.0,kr,USD,MC,1991-06-17,2016-02-02,1.000000,151,Min Chul,Jang,None,1000,1000.0,1000,5th-8th,1,Unaffiliated,506645.42,IEM X - Taipei (SC2),"[{'team': 'CJ Entus', 'history': [146396160000...",108,4725.658131,kr,USD,Unaffiliated,Unaffiliated,Cash Only,5.0,2016-02-02
2,23.0,kr,USD,MC,1991-06-17,2014-10-05,1.000000,151,Min Chul,Jang,None,1000,5000.0,5000,5th-8th,1,Unaffiliated,506645.42,WCS Europe 2014 Season 3 Premier,"[{'team': 'CJ Entus', 'history': [146396160000...",108,4688.274953,kr,USD,Unaffiliated,Unaffiliated,Cash Only,5.0,2014-10-05
3,19.0,kr,KRW,MC,1991-06-17,2011-04-09,0.000890,151,Min Chul,Jang,None,1000,10000000.0,8903,3rd-4th,1,Old Generations,506645.42,GSL World Championship Seoul 2011,"[{'team': 'CJ Entus', 'history': [146396160000...",108,4651.798318,kr,KRW,Old Generations,Old Generations,Cash Only,5.0,2011-04-09
4,17.0,kr,USD,PartinG,1994-08-24,2012-05-27,1.000000,151,Lee Sak,Won,None,1132,1200.0,1200,5th-8th,1,FXOpen e-Sports,355604.37,Red Bull Battlegrounds,"[{'team': 'Flash Wolves', 'history': [14134176...",100,3579.842121,kr,USD,FXOpen e-Sports,FXOpen e-Sports,Cash Only,4.0,2012-05-27
5,23.0,kr,USD,YongHwa,1992-01-30,2015-07-19,1.000000,151,Yong Hwa,Choi,None,2418,350.0,350,13th-16th,1,MVP,40534.16,IEM X - Shenzhen (StarCraft II),"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1030.363077,kr,USD,MVP,MVP,Cash Only,2.0,2015-07-19
6,23.0,kr,KRW,YongHwa,1992-01-30,2015-04-17,0.000914,151,Yong Hwa,Choi,None,2418,400000.0,365,25th-48th,1,MVP,40534.16,GSL Season 2 2015 (Code A),"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1029.978462,kr,KRW,MVP,MVP,Cash Only,2.0,2015-04-17
7,23.0,kr,KRW,YongHwa,1992-01-30,2015-03-22,0.000898,151,Yong Hwa,Choi,None,2418,850000.0,763,17th-24th,1,MVP,40534.16,2015 Global StarCraft II League Season 1,"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1019.773333,kr,KRW,MVP,MVP,Cash Only,2.0,2015-03-22
8,23.0,kr,USD,YongHwa,1992-01-30,2015-03-22,1.000000,151,Yong Hwa,Choi,None,2418,15.0,15,3rd-4th,1,MVP,40534.16,OlimoLeague Week #25,"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1038.952821,kr,USD,MVP,MVP,Cash Only,2.0,2015-03-22
9,23.0,kr,USD,YongHwa,1992-01-30,2015-02-26,1.000000,151,Yong Hwa,Choi,None,2418,200.0,0,WIN,7,MVP,40534.16,Showmatch mYinsanity vs. MVP,"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1039.337436,kr,USD,MVP,MVP,Winners,2.0,2015-02-26


In [7]:
df = df.set_index(np.arange(len(df))) #changing the index to something nice

In [82]:
df['num_teams'] = np.where( df['team_history']=='Unaffiliated', 0, (df.team_history.str.len()))

In [164]:
df['num_teams'] = df['num_teams'].fillna(0)

In [165]:
p = Histogram(df['Age_at_win'].dropna(), title = 'Age of players that cash in tournaments')

In [166]:
show(p)

In [167]:
p = Histogram(df['totalsc2'].unique(), title='Total Number of Cashes Per Player')

In [168]:
show(p)

In [169]:
df.Prize_USD = df.Prize_USD / df.TeamPlayers
df.Prize_USD = (df.Prize_USD.astype(int))
df[df.TotalUSDPrize < df.Prize_USD]

,Age_at_win,CountryCode,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,GameId,NameFirst,NameLast,Note,PlayerId,Prize,Prize_USD,RankText,TeamPlayers,Team_On,TotalUSDPrize,TournamentName,team_history,totalsc2,AveragePrizePerPlayer,country,currency,Team_On_Backup,teams,Rank,num_teams


In [170]:
df['AveragePrizePerPlayer']=np.where(df['totalsc2']==1,0,(df['TotalUSDPrize']-df['Prize_USD'])/(df['totalsc2']-1)) 

In [171]:
p = Histogram(df['AveragePrizePerPlayer'],title='Avg Prize Per Player')
show(p)

In [172]:
p = Histogram(np.log(df[df['AveragePrizePerPlayer']>0]['AveragePrizePerPlayer'].unique()), title='Log Average Prize Per Player')

In [173]:
show(p)

In [174]:
p = Donut(df['CountryCode'], title = "Country of Origin")
show(p)

In [175]:
df['country'] = 'Other'
top_countries = ['kr','fr','de','cn','us','ua','se','ru','pl']
for a in top_countries:
    df['country']= np.where(df['CountryCode']==a, a, df['country'])

p = Donut(df['country'], title = "Country of Origin (top 10)")
show(p)

In [176]:
p = Donut(df['CurrencyCode'], title = 'Currency of Prize Money')
show(p)

In [177]:
df['currency'] = 'Other'
top_countries = ['USD', 'KRW', 'EUR', 'CNY', 'AUD']
for a in top_countries:
    df['currency']= np.where(df['CurrencyCode']==a, a, df['currency'])

p = Donut(df['currency'], title = "Currency (Top 5)")

show(p)

In [178]:
pd.set_option('max_columns', 50)

In [179]:
df

,Age_at_win,CountryCode,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,GameId,NameFirst,NameLast,Note,PlayerId,Prize,Prize_USD,RankText,TeamPlayers,Team_On,TotalUSDPrize,TournamentName,team_history,totalsc2,AveragePrizePerPlayer,country,currency,Team_On_Backup,teams,Rank,num_teams
0,25.0,kr,KRW,MC,1991-06-17,2016-09-10,0.000905,151,Min Chul,Jang,None,1000,3000000.0,2714,17th-24th,1,CJ Entus,506645.42,GSL Season 2 2016 (Code S),"[{'team': 'CJ Entus', 'history': [146396160000...",108,4709.639439,kr,KRW,CJ Entus,CJ Entus,Cash Only,5.0
1,24.0,kr,USD,MC,1991-06-17,2016-02-02,1.000000,151,Min Chul,Jang,None,1000,1000.0,1000,5th-8th,1,Unaffiliated,506645.42,IEM X - Taipei (SC2),"[{'team': 'CJ Entus', 'history': [146396160000...",108,4725.658131,kr,USD,Unaffiliated,Unaffiliated,Cash Only,5.0
2,23.0,kr,USD,MC,1991-06-17,2014-10-05,1.000000,151,Min Chul,Jang,None,1000,5000.0,5000,5th-8th,1,Unaffiliated,506645.42,WCS Europe 2014 Season 3 Premier,"[{'team': 'CJ Entus', 'history': [146396160000...",108,4688.274953,kr,USD,Unaffiliated,Unaffiliated,Cash Only,5.0
3,19.0,kr,KRW,MC,1991-06-17,2011-04-09,0.000890,151,Min Chul,Jang,None,1000,10000000.0,8903,3rd-4th,1,Old Generations,506645.42,GSL World Championship Seoul 2011,"[{'team': 'CJ Entus', 'history': [146396160000...",108,4651.798318,kr,KRW,Old Generations,Old Generations,Cash Only,5.0
4,17.0,kr,USD,PartinG,1994-08-24,2012-05-27,1.000000,151,Lee Sak,Won,None,1132,1200.0,1200,5th-8th,1,FXOpen e-Sports,355604.37,Red Bull Battlegrounds,"[{'team': 'Flash Wolves', 'history': [14134176...",100,3579.842121,kr,USD,FXOpen e-Sports,FXOpen e-Sports,Cash Only,4.0
5,23.0,kr,USD,YongHwa,1992-01-30,2015-07-19,1.000000,151,Yong Hwa,Choi,None,2418,350.0,350,13th-16th,1,MVP,40534.16,IEM X - Shenzhen (StarCraft II),"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1030.363077,kr,USD,MVP,MVP,Cash Only,2.0
6,23.0,kr,KRW,YongHwa,1992-01-30,2015-04-17,0.000914,151,Yong Hwa,Choi,None,2418,400000.0,365,25th-48th,1,MVP,40534.16,GSL Season 2 2015 (Code A),"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1029.978462,kr,KRW,MVP,MVP,Cash Only,2.0
7,23.0,kr,KRW,YongHwa,1992-01-30,2015-03-22,0.000898,151,Yong Hwa,Choi,None,2418,850000.0,763,17th-24th,1,MVP,40534.16,2015 Global StarCraft II League Season 1,"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1019.773333,kr,KRW,MVP,MVP,Cash Only,2.0
8,23.0,kr,USD,YongHwa,1992-01-30,2015-03-22,1.000000,151,Yong Hwa,Choi,None,2418,15.0,15,3rd-4th,1,MVP,40534.16,OlimoLeague Week #25,"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1038.952821,kr,USD,MVP,MVP,Cash Only,2.0
9,23.0,kr,USD,YongHwa,1992-01-30,2015-02-26,1.000000,151,Yong Hwa,Choi,None,2418,200.0,0,WIN,7,MVP,40534.16,Showmatch mYinsanity vs. MVP,"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1039.337436,kr,USD,MVP,MVP,Winners,2.0


In [180]:
# p = figure()
# p.circle(x = df['EndDate'], y =df['Prize_USD'])
# p.xaxis.formatter = dttf.months

In [181]:
# show(p) #need to figure out date/Time on Bokeh

In [182]:
df['Team_On_Backup']=df['Team_On']

In [183]:
df['Team_On']=np.where(df['Team_On']==1, "Unaffiliated", df['Team_On']) #cleaning

In [184]:
p = Donut(df['Team_On'], title = 'Distribution of Winners')
show(p)

In [185]:
a = df.Team_On.value_counts().nlargest(26)
top_teams = list(a.index)
df['teams'] = 'Other'
for a in top_teams:
    df['teams']= np.where(df['Team_On']==a, a, df['teams'])

p = Donut(df['teams'], title = "Teams (Top 25)")
show(p)

In [186]:
df['RankText'].unique()

array(['17th-24th', '5th-8th', '3rd-4th', '13th-16th', '25th-48th', 'WIN',
       '2nd', '3rd', '1st', '25th-32nd', 'Round 1', 'Code S', 'Round 2',
       'Round 3', '9th-11th', '4th', '5th-6th', '9th-16th', '11th-14th',
       'LOSE', '17th-32nd', '7th-8th', '-', '17th-20th', '9th-12th',
       '1 Win', '6th-8th', 'See Note', '9th', '7th', '8th', '5 Wins',
       '31st-60th', '7th-12th', '5th', '6th', '33rd-64th', '3 Wins',
       '9th-10th', '29th-36th', '1st-8th', '21st-24th', '11th-12th',
       '2 Wins', '13th', '8 Wins', '19th-26th', '15th-18th', '27th-32nd',
       '10th-12th', '10th-11th', '5th-12th', '7th-10th', '13th-14th',
       '37th-44th', '11th', '25th-28th', '10th', 'DISQUALIFIED', '12th',
       '14th', '15th', '1st-4th', '4 Wins', '7 Wins'], dtype=object)

In [187]:
#Changing Ranks to Categoricals
df['Rank'] = 'Cash Only'

win = ['WIN','1st','1st-4th']
for a in win:
    df['Rank']=np.where(df['RankText'] == a, 'Winners', df['Rank'])
    
second = ['2nd', '7 Wins','8 Wins','6 Wins', '3rd']
for a in second:
    df['Rank']=np.where(df['RankText'] == a, 'Runners Up', df['Rank'])

In [188]:
p = Donut(df['Rank'], title = 'Distribution of Winners')
show(p)

In [189]:
df_clean = df
df_messy = df
df_messy.to_json('messy.json')

In [190]:
df_clean

,Age_at_win,CountryCode,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,GameId,NameFirst,NameLast,Note,PlayerId,Prize,Prize_USD,RankText,TeamPlayers,Team_On,TotalUSDPrize,TournamentName,team_history,totalsc2,AveragePrizePerPlayer,country,currency,Team_On_Backup,teams,Rank,num_teams
0,25.0,kr,KRW,MC,1991-06-17,2016-09-10,0.000905,151,Min Chul,Jang,None,1000,3000000.0,2714,17th-24th,1,CJ Entus,506645.42,GSL Season 2 2016 (Code S),"[{'team': 'CJ Entus', 'history': [146396160000...",108,4709.639439,kr,KRW,CJ Entus,CJ Entus,Cash Only,5.0
1,24.0,kr,USD,MC,1991-06-17,2016-02-02,1.000000,151,Min Chul,Jang,None,1000,1000.0,1000,5th-8th,1,Unaffiliated,506645.42,IEM X - Taipei (SC2),"[{'team': 'CJ Entus', 'history': [146396160000...",108,4725.658131,kr,USD,Unaffiliated,Unaffiliated,Cash Only,5.0
2,23.0,kr,USD,MC,1991-06-17,2014-10-05,1.000000,151,Min Chul,Jang,None,1000,5000.0,5000,5th-8th,1,Unaffiliated,506645.42,WCS Europe 2014 Season 3 Premier,"[{'team': 'CJ Entus', 'history': [146396160000...",108,4688.274953,kr,USD,Unaffiliated,Unaffiliated,Cash Only,5.0
3,19.0,kr,KRW,MC,1991-06-17,2011-04-09,0.000890,151,Min Chul,Jang,None,1000,10000000.0,8903,3rd-4th,1,Old Generations,506645.42,GSL World Championship Seoul 2011,"[{'team': 'CJ Entus', 'history': [146396160000...",108,4651.798318,kr,KRW,Old Generations,Old Generations,Cash Only,5.0
4,17.0,kr,USD,PartinG,1994-08-24,2012-05-27,1.000000,151,Lee Sak,Won,None,1132,1200.0,1200,5th-8th,1,FXOpen e-Sports,355604.37,Red Bull Battlegrounds,"[{'team': 'Flash Wolves', 'history': [14134176...",100,3579.842121,kr,USD,FXOpen e-Sports,FXOpen e-Sports,Cash Only,4.0
5,23.0,kr,USD,YongHwa,1992-01-30,2015-07-19,1.000000,151,Yong Hwa,Choi,None,2418,350.0,350,13th-16th,1,MVP,40534.16,IEM X - Shenzhen (StarCraft II),"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1030.363077,kr,USD,MVP,MVP,Cash Only,2.0
6,23.0,kr,KRW,YongHwa,1992-01-30,2015-04-17,0.000914,151,Yong Hwa,Choi,None,2418,400000.0,365,25th-48th,1,MVP,40534.16,GSL Season 2 2015 (Code A),"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1029.978462,kr,KRW,MVP,MVP,Cash Only,2.0
7,23.0,kr,KRW,YongHwa,1992-01-30,2015-03-22,0.000898,151,Yong Hwa,Choi,None,2418,850000.0,763,17th-24th,1,MVP,40534.16,2015 Global StarCraft II League Season 1,"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1019.773333,kr,KRW,MVP,MVP,Cash Only,2.0
8,23.0,kr,USD,YongHwa,1992-01-30,2015-03-22,1.000000,151,Yong Hwa,Choi,None,2418,15.0,15,3rd-4th,1,MVP,40534.16,OlimoLeague Week #25,"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1038.952821,kr,USD,MVP,MVP,Cash Only,2.0
9,23.0,kr,USD,YongHwa,1992-01-30,2015-02-26,1.000000,151,Yong Hwa,Choi,None,2418,200.0,0,WIN,7,MVP,40534.16,Showmatch mYinsanity vs. MVP,"[{'team': 'MVP', 'history': [1406764800000, 14...",40,1039.337436,kr,USD,MVP,MVP,Winners,2.0


In [191]:
to_drop = ['CountryCode','CurrencyCode','CurrentHandle','ExchangeRate','GameId','NameFirst','NameLast','Note',
            'PlayerId','Prize','RankText','TeamPlayers','Team_On','TournamentName','team_history',
            'Team_On_Backup','DOB','EndDate','TotalUSDPrize', 'Prize_USD']
df_dropped = df_clean.drop(to_drop, axis=1)
df_dropped['totalsc2']=df_dropped['totalsc2']-1

In [192]:
df_dropped.head()

,Age_at_win,totalsc2,AveragePrizePerPlayer,country,currency,teams,Rank,num_teams
0,25.0,107,4709.639439,kr,KRW,CJ Entus,Cash Only,5.0
1,24.0,107,4725.658131,kr,USD,Unaffiliated,Cash Only,5.0
2,23.0,107,4688.274953,kr,USD,Unaffiliated,Cash Only,5.0
3,19.0,107,4651.798318,kr,KRW,Old Generations,Cash Only,5.0
4,17.0,99,3579.842121,kr,USD,FXOpen e-Sports,Cash Only,4.0


In [193]:
df_dums = pd.get_dummies(data=df_dropped, columns = ['Rank','currency','teams','country'])
df_dums['Age_at_win'] = df_dums['Age_at_win'].fillna(df_dums['Age_at_win'].mean())

In [194]:
df_dums.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19969 entries, 0 to 19968
Data columns (total 50 columns):
Age_at_win                             19969 non-null float64
totalsc2                               19969 non-null int64
AveragePrizePerPlayer                  19969 non-null float64
num_teams                              19969 non-null float64
Rank_Cash Only                         19969 non-null uint8
Rank_Runners Up                        19969 non-null uint8
Rank_Winners                           19969 non-null uint8
currency_AUD                           19969 non-null uint8
currency_CNY                           19969 non-null uint8
currency_EUR                           19969 non-null uint8
currency_KRW                           19969 non-null uint8
currency_Other                         19969 non-null uint8
currency_USD                           19969 non-null uint8
teams_CJ Entus                         19969 non-null uint8
teams_Dead Pixels                      19969 

In [195]:
#prep for classification
df_class = df_dums.drop(['Rank_Runners Up','Rank_Cash Only'], axis=1)
#test to see if there is leakage in the average/total:
df_class = df_class.drop(['totalsc2', 'AveragePrizePerPlayer'], axis =1)
y = df_class.pop('Rank_Winners')
df_class.head()

,Age_at_win,num_teams,currency_AUD,currency_CNY,currency_EUR,currency_KRW,currency_Other,currency_USD,teams_CJ Entus,teams_Dead Pixels,teams_ESC Gaming,teams_Evil Geniuses,teams_FXOpen e-Sports,teams_Incredible Miracle,teams_Invictus Gaming,teams_Jin Air Green Wings,teams_KT Rolster,teams_MVP,teams_Millenium,teams_Old Generations,teams_Other,teams_PSISTORM Gaming,teams_Prime,teams_ROOT Gaming,teams_SK Telecom T1,teams_SlayerS,teams_StarCraft II (Samsung Galaxy),teams_StarTale,teams_Team Acer,teams_Team Empire,teams_Team Liquid,teams_Team SCV Life,teams_Unaffiliated,teams_mYinsanity,teams_mousesports,country_Other,country_cn,country_de,country_fr,country_kr,country_pl,country_ru,country_se,country_ua,country_us
0,25.0,5.0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,24.0,5.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
2,23.0,5.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
3,19.0,5.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,17.0,4.0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [196]:
X, X_finaltest, y, y_final_test = train_test_split(df_class,y, random_state=314)

In [197]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [198]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

#Cross-Validation
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Train Crossval Accuracy: " + str(scores))
print("Predicted Accuracy: "+ str(sum(scores)/5))

preds = clf.predict(X_test)
score = clf.score(X_test,y_test)
print('Test Accuracy: ' + str(score))

Train Crossval Accuracy: [ 0.73386738  0.73208723  0.73107747  0.73374889  0.73508459]
Predicted Accuracy: 0.733173111789
Test Accuracy: 0.725961538462


In [199]:
clf = GradientBoostingClassifier(learning_rate=.01, max_features=10,n_estimators=1000)
clf.fit(X_train, y_train)

#Cross-Validation
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Train Crossval Accuracy: " + str(scores))
print("Predicted Accuracy: "+ str(sum(scores)/5))

preds = clf.predict(X_test)
score = clf.score(X_test,y_test)
print('Test Accuracy: ' + str(score))

Train Crossval Accuracy: [ 0.73876279  0.73386738  0.730187    0.73998219  0.73597507]
Predicted Accuracy: 0.735754886004
Test Accuracy: 0.735844017094


In [200]:
#Prep for Regression:
to_drop = ['CountryCode','CurrencyCode','CurrentHandle','ExchangeRate','GameId','NameFirst','NameLast','Note',
            'PlayerId','Prize','RankText','TeamPlayers','Team_On','TournamentName','team_history',
            'Team_On_Backup','DOB','EndDate','TotalUSDPrize']
df_dropped = df_clean.drop(to_drop, axis=1)
df_dums = pd.get_dummies(data=df_dropped, columns = ['Rank','currency','teams','country'])
df_dums['Age_at_win'] = df_dums['Age_at_win'].fillna(df_dums['Age_at_win'].mean())
df_reg = df_dums.drop('AveragePrizePerPlayer', axis = 1)
df_reg['Int']=0
y = df_reg.pop('Prize_USD')

In [47]:
X, X_finaltest, y, y_final_test = train_test_split(df_class,y, random_state=314)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [520]:
reg = LinearRegression()
reg.fit(X_train, y_train)
reg.coef_

#Cross-Validation
scores = cross_val_score(reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print("Train MSE: " + str(scores))
print("Predicted MSE: "+ str(sum(scores)/5))

preds = reg.predict(X_test)
score = reg.score(X_test,y_test)
print('Test Accuracy: ' + str(score))

Train MSE: [-22595654.44590788 -21882040.0098787  -24139291.82018976
  -7816643.63063937 -31825078.717377  ]
Predicted MSE: -21651741.7248
Test Accuracy: 0.0637680148848
